# 准备知识

1. 动态网页的**```爬取思路```**与静态网页**```基本一致```**

|**步骤**|任务|说明|方法|
|:----:|:----:|:-----|:----:|
|**1**|**生成网址**|分析<mark>网址规律</mark>，批量生成网址|for循环，format函数|
|2|请求+获取网页数据|模拟人工打开网页，并将网页存储为数据对象|requests包|
|**3**|**解析数据**|分析数据规律，整理出所需字段|pyquery包，**<mark>json包</mark>**<br>（还有lxml和beautifulsoup4）|
|4|存储数据|使用csv包将数据存储到csv文件中|csv包|
|5|批量爬取|对所有网址循环步骤2-4|for循环|

2. **```动态与静态的区别```**
- （1）<u>网址规律</u>：
    - 翻页时，动态网页的**<mark>网址不变化</mark>**
    - 无法在网页源代码直接看到内容<br>
<br>
- （2）<u>解析数据</u>：
    - 不是网页源代码的字符串html，而是**<mark>字典列表json</mark>**

# 生成网址

## 分析网址规律

1. **```任务对象```**：
    - https://item.jd.com/100002781562.html#comment<br>
<br>
<br>
2. **```基本情况```**：
    - 不论怎么翻页，**<u>```网址都没有变化```</u>**
    - 每页10个评论，最多翻到100页

3. **```原因分析及应对```**：
    - 真正的数据存在别的网址中，当我们点击对应的页数时，它才被“调用”，然后显示在我们的浏览器界面中
    - **<u>```用开发者工具中的network，观察翻页时文件的变化，找到存储“评论”的文件，再查看headers中的request url，这就是真正的数据网址。```</u>**（确认评论数据存储在request url中）

4. **```观察规律```**：
- **<u>```原网址```</u>**：
    * https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=0&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=
    * https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=1&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=
    * https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=2&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=
    * ……
    * https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=98&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=
    * https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=99&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=<br>
<br>
- **<u>```总结规律```</u>**：
    * 第1页，page为0
    * 第2页，page为1
    * 第3页，page为2
    * ……
    * 第99页，page为98
    * 第100页，page为99
<br>
<br>
    * <mark><big>第p页，page为(p-1)</big></mark>

## 批量生成网址

In [1]:
# 每个商品有自己的唯一id，例如100002781562、1540112、10080196292423等
product_id = '100002781562'

# format函数设置product_id和page
template = 'https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId={product_id}&score=0&sortType=5&page={page}&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield='

# 循环生成网址，并存入网址列表中
url_list = []
for p in range(1,101):
    url = template.format(product_id = product_id, page=(p-1))   # 分别填入product_id和page
    url_list.append(url)

In [2]:
# 查看url_list情况（获得10页的网址）
url_list

['https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=0&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=',
 'https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=1&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=',
 'https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=2&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=',
 'https://api.m.jd.com/?appid=item-v3&functionId=pc_club_prod

In [3]:
# 函数：生成网址  generate_url_list(product_id, max_page)
# 参数说明：product_id为商品id，max_page为最大页数
# 返回值：url_list为网址列表
def generate_url_list(product_id, max_page):
    url_list = []
    template = 'https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId={product_id}&score=0&sortType=5&page={page}&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield='
    for p in range(1,max_page+1):        # range取1到100，即p循环1到100页
        url = template.format(product_id = product_id, page=(p-1))
        url_list.append(url)
    return url_list

In [4]:
# 调用函数  generate_url_list(product_id, max_page)
url_list_tmp1 = generate_url_list(product_id=100002781562, max_page=5)
url_list_tmp2 = generate_url_list(product_id=1540112, max_page=10)

print('商品id为100002781562的5条数据网址：',url_list_tmp1,'\n')
print('商品id为1540112的10条数据网址：',url_list_tmp2)

商品id为100002781562的5条数据网址： ['https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=0&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=', 'https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=1&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=', 'https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=2&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=', 'https://api.m.jd.com/?appid=item-v3&

# 请求+获取网页数据

- 以第1页为例，来请求并获取网页数据
- 例子：https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=0&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=

In [5]:
import requests   # 导入requests包

url = 'https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=0&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield='
resp = requests.get(url)   # 用get向服务器请求获取数据
resp # 查看状态码，2开头代表访问成功，4开头代表不成功

<Response [200]>

In [6]:
resp.text   # 查看返回内容

'{"jwotestProduct":null,"score":0,"comments":[{"id":20711328678,"guid":"1cd1799369aac08ae868dd60d9ac59ec","content":"刚收到拆了一把，质感很不错，物美价廉，尺寸也刚好，大品牌还是值得信赖的，希望再出一些尺寸小一点的自动儿童伞，而且价格也不要很贵","creationTime":"2024-04-19 16:41:07","isDelete":false,"isTop":false,"userImageUrl":"storage.360buyimg.com/i.imageUpload/6a645f3731363536623835363136376431343731313238343432363130_sma.jpg","topped":0,"replyCount":0,"score":5,"imageStatus":1,"usefulVoteCount":1,"userClient":4,"discussionId":1502264719,"imageCount":2,"anonymousFlag":1,"plusAvailable":201,"mobileVersion":"12.6.2","images":[{"id":-1748611282,"imgUrl":"//img30.360buyimg.com/n0/s128x96_jfs/t1/85334/15/45031/39302/66222e22Fb1050f49/2a33f837949c7857.jpg","imgTitle":"","status":0},{"id":-1748611281,"imgUrl":"//img30.360buyimg.com/n0/s128x96_jfs/t1/197460/36/40712/31051/66222e23F6e726be4/1bb173404cda1ca9.jpg","imgTitle":"","status":0}],"mergeOrderStatus":2,"productColor":"【全自动常规款】藏青","productSize":"","textIntegral":20,"imageIntegral":0,"ownId":1000011

In [7]:
raw_comments = resp.text  # 将内容放入raw_comments中，类型为str
type(raw_comments)

str

In [8]:
# 函数：获得json  get_json(url)
# 参数说明：url为单个网址
# 返回值：raw_comments为原始的评论数据
def get_json(url):
    resp = requests.get(url)
    raw_comments = resp.text
    return raw_comments

In [9]:
# 调用函数  get_json(url) 以第8页为例
url_tmp = 'https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=7&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield='  # 第8页
raw_comments_tmp = get_json(url_tmp)
raw_comments_tmp

'{"jwotestProduct":null,"score":0,"comments":[{"id":20684990837,"guid":"c5ef6dae44bde47002d5a673eba36a2e","content":"此款天堂伞包装较好，适合两个人，伞面较大，使用较顺滑，看着质量不错，价格还可以，总体较满意，防雨效果不错，斜阳效果稍差，京东物流快捷。","creationTime":"2024-04-12 10:42:22","isDelete":false,"isTop":false,"userImageUrl":"storage.360buyimg.com/i.imageUpload/c0bcd6f1bed5c3b730373032303131343539353531383131393632_sma.jpg","topped":0,"replyCount":0,"score":5,"imageStatus":1,"usefulVoteCount":0,"userClient":2,"discussionId":1497829962,"imageCount":2,"anonymousFlag":1,"plusAvailable":201,"mobileVersion":"12.6.0","images":[{"id":-1757490786,"imgUrl":"//img30.360buyimg.com/n0/s128x96_jfs/t1/147818/22/43702/179652/66189f89F871b4f83/0fb7e52067ba765c.jpg","imgTitle":"","status":0},{"id":-1757490785,"imgUrl":"//img30.360buyimg.com/n0/s128x96_jfs/t1/246608/35/6758/220656/66189f8eFf4e5a3f8/67244f0eb37d143b.jpg","imgTitle":"","status":0}],"mergeOrderStatus":2,"productColor":"【全自动常规款】兰灰","productSize":"","textIntegral":20,"imageIntegral":20,"ownId":1000

# 解析数据

观察raw_comments数据结构
- https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=100002781562&score=0&sortType=5&page=0&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield='
1. comments里有多个字典，每一个字典对应一个用户评论
2. key：content内容、creationTime时间、plusAvailable会员身份、nickname用户名、score评分、usefulVoteCount点赞数、replyCount回复数、productColor颜色、productSize尺码等

In [10]:
import json

json.loads(raw_comments)   # 如果raw_comments是字典列表，那么能正常loads；如果【不是规范的字典列表】，那么会报错

{'jwotestProduct': None,
 'score': 0,
 'comments': [{'id': 20711328678,
   'guid': '1cd1799369aac08ae868dd60d9ac59ec',
   'content': '刚收到拆了一把，质感很不错，物美价廉，尺寸也刚好，大品牌还是值得信赖的，希望再出一些尺寸小一点的自动儿童伞，而且价格也不要很贵',
   'creationTime': '2024-04-19 16:41:07',
   'isDelete': False,
   'isTop': False,
   'userImageUrl': 'storage.360buyimg.com/i.imageUpload/6a645f3731363536623835363136376431343731313238343432363130_sma.jpg',
   'topped': 0,
   'replyCount': 0,
   'score': 5,
   'imageStatus': 1,
   'usefulVoteCount': 1,
   'userClient': 4,
   'discussionId': 1502264719,
   'imageCount': 2,
   'anonymousFlag': 1,
   'plusAvailable': 201,
   'mobileVersion': '12.6.2',
   'images': [{'id': -1748611282,
     'imgUrl': '//img30.360buyimg.com/n0/s128x96_jfs/t1/85334/15/45031/39302/66222e22Fb1050f49/2a33f837949c7857.jpg',
     'imgTitle': '',
     'status': 0},
    {'id': -1748611281,
     'imgUrl': '//img30.360buyimg.com/n0/s128x96_jfs/t1/197460/36/40712/31051/66222e23F6e726be4/1bb173404cda1ca9.jpg',
     'imgTi

In [11]:
type(json.loads(raw_comments))

dict

In [12]:
comments = json.loads(raw_comments)['comments']   # 取comments键的值，并存入comments对象中
comments

[{'id': 20711328678,
  'guid': '1cd1799369aac08ae868dd60d9ac59ec',
  'content': '刚收到拆了一把，质感很不错，物美价廉，尺寸也刚好，大品牌还是值得信赖的，希望再出一些尺寸小一点的自动儿童伞，而且价格也不要很贵',
  'creationTime': '2024-04-19 16:41:07',
  'isDelete': False,
  'isTop': False,
  'userImageUrl': 'storage.360buyimg.com/i.imageUpload/6a645f3731363536623835363136376431343731313238343432363130_sma.jpg',
  'topped': 0,
  'replyCount': 0,
  'score': 5,
  'imageStatus': 1,
  'usefulVoteCount': 1,
  'userClient': 4,
  'discussionId': 1502264719,
  'imageCount': 2,
  'anonymousFlag': 1,
  'plusAvailable': 201,
  'mobileVersion': '12.6.2',
  'images': [{'id': -1748611282,
    'imgUrl': '//img30.360buyimg.com/n0/s128x96_jfs/t1/85334/15/45031/39302/66222e22Fb1050f49/2a33f837949c7857.jpg',
    'imgTitle': '',
    'status': 0},
   {'id': -1748611281,
    'imgUrl': '//img30.360buyimg.com/n0/s128x96_jfs/t1/197460/36/40712/31051/66222e23F6e726be4/1bb173404cda1ca9.jpg',
    'imgTitle': '',
    'status': 0}],
  'mergeOrderStatus': 2,
  'productColor': '【全

In [13]:
# comments为字典列表，每一个字典是一条用户评论
comments[0].keys() # 查看字典的key

dict_keys(['id', 'guid', 'content', 'creationTime', 'isDelete', 'isTop', 'userImageUrl', 'topped', 'replyCount', 'score', 'imageStatus', 'usefulVoteCount', 'userClient', 'discussionId', 'imageCount', 'anonymousFlag', 'plusAvailable', 'mobileVersion', 'images', 'mergeOrderStatus', 'productColor', 'productSize', 'textIntegral', 'imageIntegral', 'ownId', 'ownType', 'extMap', 'location', 'status', 'referenceId', 'referenceTime', 'nickname', 'replyCount2', 'userImage', 'orderId', 'integral', 'productSales', 'referenceImage', 'referenceName', 'firstCategory', 'secondCategory', 'thirdCategory', 'aesPin', 'days', 'afterDays'])

- 我们只取其中的content内容、creationTime时间、plusAvailable会员身份、nickname用户名、score评分、usefulVoteCount点赞数、replyCount回复数、productColor颜色、productSize尺码等

In [14]:
data_list = []                               # data_list为空列表，用于存储评论信息

for comment in comments:
    data = {}                                # data为空字典
    
    data['content']=comment['content']   # 取content键的值，放入data中的content键值对
    data['creationTime']=comment.get('creationTime')
    data['nickname']=comment.get('nickname')
    data['plusAvailable']=comment.get('plusAvailable')
    data['score']=comment.get('score')
    data['usefulVoteCount']=comment.get('usefulVoteCount')
    data['replyCount']=comment.get('replyCount')
    data['productColor']=comment.get('productColor')
    data['productSize']=comment.get('productSize')
    
    data_list.append(data)

print(data_list)
print(len(data_list))

[{'content': '很可惜没有买到黑色的，这种灰色的有点不太搭配，雨伞本身很好，很有型，方便整理，特意买了两把雨伞对比了下，这个雨伞最终胜出了，除了颜色这伞没其他可挑剔的。', 'creationTime': '2023-07-29 09:25:45', 'nickname': '笑***凋', 'plusAvailable': 201, 'score': 5, 'usefulVoteCount': 1, 'replyCount': 0, 'productColor': '【全自动常规款】黎色', 'productSize': ''}, {'content': '尺寸大小：一个人用用着挺好\n遮阳效果：黑胶 遮雨两用 方便\n防雨效果：没有漏点  效果挺好\n其他特色：质量很好 做工好  打开关闭 很轻松    价格一般般', 'creationTime': '2023-07-24 09:38:54', 'nickname': '伊***护', 'plusAvailable': 201, 'score': 5, 'usefulVoteCount': 2, 'replyCount': 0, 'productColor': '【全自动黑胶款】 黛蓝', 'productSize': ''}, {'content': '天堂伞收到了，质量不错，做工精细，款式时尚大方收到货后试了一下，很不错，质量很好，值得拥有！五星好评！趁着优惠，赶紧入手，是正品，包装也非常的好，没有破损，质量也是没得说，棒?', 'creationTime': '2023-07-13 16:08:06', 'nickname': '知***4', 'plusAvailable': 201, 'score': 5, 'usefulVoteCount': 1, 'replyCount': 0, 'productColor': '【全自动常规款兰灰', 'productSize': ''}, {'content': '遮阳效果：目前没在太阳下打过，等试后再做评价\n防雨效果：伞布很厚实，效果不错\n尺寸大小：挺合适\n其他特色：伞界知名品牌，老品牌了，信得过质量，很有质感，设计也很合理。', 'creationTime': '2023-07-21 01:06:13', 'nickname': 'A**

In [15]:
# 函数：解析数据  extract_comments(raw_comments)
# 参数说明：raw_comments为初始的评论列表
# 返回值：data_list为整理后的评论列表
def extract_comments(raw_comments):
    data_list = []
    
    comments = json.loads(raw_comments)['comments']   
    
    for comment in comments:    # 将一条条评论写入data_list中
        data = {}
        data['content']=comment.get('content')
        data['creationTime']=comment.get('creationTime')
        data['nickname']=comment.get('nickname')
        data['plusAvailable']=comment.get('plusAvailable')
        data['score']=comment.get('score')
        data['usefulVoteCount']=comment.get('usefulVoteCount')
        data['replyCount']=comment.get('replyCount')
        data['productColor']=comment.get('productColor')
        data['productSize']=comment.get('productSize')
        data_list.append(data)
    
    return data_list    

In [16]:
# 调用函数   extract_comments(raw_comments)
comments = extract_comments(raw_comments)
print(data_list)
print('评论数量为：', len(data_list))

[{'content': '很可惜没有买到黑色的，这种灰色的有点不太搭配，雨伞本身很好，很有型，方便整理，特意买了两把雨伞对比了下，这个雨伞最终胜出了，除了颜色这伞没其他可挑剔的。', 'creationTime': '2023-07-29 09:25:45', 'nickname': '笑***凋', 'plusAvailable': 201, 'score': 5, 'usefulVoteCount': 1, 'replyCount': 0, 'productColor': '【全自动常规款】黎色', 'productSize': ''}, {'content': '尺寸大小：一个人用用着挺好\n遮阳效果：黑胶 遮雨两用 方便\n防雨效果：没有漏点  效果挺好\n其他特色：质量很好 做工好  打开关闭 很轻松    价格一般般', 'creationTime': '2023-07-24 09:38:54', 'nickname': '伊***护', 'plusAvailable': 201, 'score': 5, 'usefulVoteCount': 2, 'replyCount': 0, 'productColor': '【全自动黑胶款】 黛蓝', 'productSize': ''}, {'content': '天堂伞收到了，质量不错，做工精细，款式时尚大方收到货后试了一下，很不错，质量很好，值得拥有！五星好评！趁着优惠，赶紧入手，是正品，包装也非常的好，没有破损，质量也是没得说，棒?', 'creationTime': '2023-07-13 16:08:06', 'nickname': '知***4', 'plusAvailable': 201, 'score': 5, 'usefulVoteCount': 1, 'replyCount': 0, 'productColor': '【全自动常规款兰灰', 'productSize': ''}, {'content': '遮阳效果：目前没在太阳下打过，等试后再做评价\n防雨效果：伞布很厚实，效果不错\n尺寸大小：挺合适\n其他特色：伞界知名品牌，老品牌了，信得过质量，很有质感，设计也很合理。', 'creationTime': '2023-07-21 01:06:13', 'nickname': 'A**

# 存储数据

In [17]:
import csv     # 导入csv包

# 打开文件
file = open('output/umbrella_100002781562.csv', 'a+', encoding='utf-8', newline='') # 文件名为umbrella_100002781562.csv，模式为a+，如果已有文件，在末尾追加，如果没，则生成新文件；编码为utf-8，需要区分换行符
fieldnames = ['content', 'creationTime', 'nickname', 'plusAvailable', 'score', 'usefulVoteCount', 'replyCount', 'productColor', 'productSize']  # 设置标题行fieldnames
writer = csv.DictWriter(file, fieldnames=fieldnames)    # 要求以字典的形式写入数据，fieldnames注明字典中键的名称，也就是评论内容、时间、用户名等
writer.writeheader()  # 将fieldnames设置的标题key写入首行

# 循环写入字典列表：因为有多条评论，需要一行行写入
for data in data_list:
    writer.writerow(data)  # 写入一行评论数据
    
file.close() # 关闭文件

# 批量爬取

<b><mark>下面对所有网址url_list循环步骤2-4</mark><b>

|**步骤**|任务|函数|输入参数|返回值|
|:----:|:-----|:-----|:-----|:-----|
|1|生成网址|generate_url_list(product_id, max_page)|product_id为商品id，max_page为最大页数|url_list网址列表|
|**2**|**请求+获取网页数据**|get_json(url)|单个网址|raw_comments原始的评论数据|
|**3**|**解析数据**|extract_comments(raw_comments)|（单个网址）raw_comments原始的评论数据|data_list评论的字典列表|
|**4**|**存储数据**|--|--|--|
|5|批量爬取|--|--|--|

<br>
<b>与静态网页思路一致，直接写主函数<b>

写主函数前，还有个**```time知识点```**
- 如果我们访问速度过快，也可能会被服务器反爬，如果我们**```间隔一段时间```**再访问，则能降低被反爬的概率

In [18]:
import time
import random

#random()随机返回[0,1)范围内的实数
print(random.random()*3)
print(random.random()*3)

# 休息对应的时间
time.sleep(random.random()*3)

2.5563488599998223
1.6134625290089333


In [19]:
# 函数：爬虫主函数  main(product_id, max_page, filename)
# 参数说明：product_id为商品id，max_page为最大页数，filename为文件名称
# 仅执行命令，不返回任何值
def main(product_id, max_page, filename):      
    print('开始采集商品id为{product_id}的商品评论！'.format(product_id = product_id))        
    
    # 生成所有网址url_list
    url_list = generate_url_list(product_id, max_page)   
    
    # 打开文件
    file = open(filename, 'a+', encoding='utf-8', newline='')  
    fieldnames = ['content', 'creationTime', 'nickname', 'plusAvailable', 'score', 'usefulVoteCount', 'replyCount', 'productColor', 'productSize']
    writer = csv.DictWriter(file, fieldnames=fieldnames)    
    writer.writeheader() 

    # 对所有网址url_list循环步骤2-4
    for url in url_list:
        print('正在采集：{url}'.format(url=url))
        raw_comments = get_json(url)                        # 【步骤2：请求+获取网页数据】
        time.sleep(random.random()*3)                  # 间隔不定长时间
        data_list = extract_comments(raw_comments) # 【步骤3：解析数据】
        for data in data_list:              # 【步骤4：存储数据】
            writer.writerow(data)

    file.close()

    print('采集完毕！')

In [ ]:
main(product_id = 100002781562, max_page=5, filename='output/umbrella_5.csv')

In [20]:
main(product_id = 1540112, max_page=10, filename='output/florida_10.csv')

开始采集商品id为1540112的商品评论！
正在采集：https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=1540112&score=0&sortType=5&page=0&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=
正在采集：https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=1540112&score=0&sortType=5&page=1&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=
正在采集：https://api.m.jd.com/?appid=item-v3&functionId=pc_club_productPageComments&client=pc&clientVersion=1.0.0&t=1691385196787&loginType=3&uuid=122270672.16913846884751013594747.1691384688.1691384688.1691384688.1&productId=1540112&score=0&sortType=5&page=2&pageSize=10&isShadowSku=0&rid=0&fold=1&bbtf=&shield=
正在采集：https://api.m.jd.com/?appid=item-v3&functionI

- 我们读取下刚才生成的文件

In [21]:
import pandas as pd

In [22]:
pd_reader = pd.read_csv('output/florida_10.csv')
pd_reader

,content,creationTime,nickname,plusAvailable,score,usefulVoteCount,replyCount,productColor,productSize
0,六神驱蚊花露水，每年夏季必备，清新花香，喷在身上香味好闻，家宝特别喜欢，驱蚊效果非常好，远离...,2023-07-26 17:03:29,L***n,201,5,1,0,【驱蚊】便携喷雾 清新花香*1瓶,NaN
1,很好，很嗨很好、很凉快。用这是谁很不错的。酒精多多的，凉感，还是有一定的刺激。要注意不能带上...,2023-07-26 09:18:18,李***5,201,5,1,0,【爆凉】便携喷雾 冰爽提神*1瓶,NaN
2,六神花露水夏季必备的花露水，晚上遛娃必须带上，防止招蚊子，这款花露水用了好久，京东商城购买。...,2023-08-04 12:34:17,y***6,201,5,1,0,【驱蚊】便携喷雾 冰莲香型*1瓶,NaN
3,一直用的这个牌子，很好用，很喜欢，价格也实惠，味道很清香，也不是那么浓，夏天蚊子太多，必须要...,2023-07-19 05:12:38,独***9,201,5,0,0,【驱蚊】便携喷雾 清新花香*3瓶,NaN
4,六神驱蚊花露水真的太神奇了。看了很多驱蚊花露水，六神的特别好用。喷上花露水之后，蚊子躲得远远...,2023-07-28 11:08:17,蓝***咩,201,5,0,0,【驱蚊】便携喷雾 清新花香*3瓶,NaN
...,...,...,...,...,...,...,...,...,...
95,这种驱蚊花露水，这几年一直在使用，驱蚊效果很好，喷上后，蚊子?只会在身边飞，却不敢落在身上。...,2023-07-09 14:14:09,s***m,201,5,0,0,【驱蚊】便携喷雾 清新花香*1瓶,NaN
96,老牌子国货了，从小用到大，品质一直未变，还是原来的配方，驱蚊效果杠杠的好，出门旅游避免的蚊虫...,2023-07-29 09:56:41,大***2,201,5,0,0,【驱蚊】便携喷雾 冰莲香型*3瓶,NaN
97,物流速度很快，价格也很实惠。喷后很凉爽很舒服，止痒效果也很不错，驱蚊也还行，闻起来也很香包装...,2023-07-09 22:33:43,A***神,201,5,0,0,【止痒】便携喷雾 花漾清新*1瓶,NaN
98,快递收到了，快递员很辛苦的，送货上门，谢谢了！这个驱蚊花露水，可以防蚊子咬着，咬着了也可以喷...,2023-07-24 17:43:25,g***8,201,5,0,0,【驱蚊】便携喷雾 清新花香*1瓶,NaN


In [23]:
pd_reader.shape    # 查看行数，列数

(100, 9)

In [24]:
pd_reader.head(10)    # 查看前几行信息，默认为5

,content,creationTime,nickname,plusAvailable,score,usefulVoteCount,replyCount,productColor,productSize
0,六神驱蚊花露水，每年夏季必备，清新花香，喷在身上香味好闻，家宝特别喜欢，驱蚊效果非常好，远离...,2023-07-26 17:03:29,L***n,201,5,1,0,【驱蚊】便携喷雾 清新花香*1瓶,NaN
1,很好，很嗨很好、很凉快。用这是谁很不错的。酒精多多的，凉感，还是有一定的刺激。要注意不能带上...,2023-07-26 09:18:18,李***5,201,5,1,0,【爆凉】便携喷雾 冰爽提神*1瓶,NaN
2,六神花露水夏季必备的花露水，晚上遛娃必须带上，防止招蚊子，这款花露水用了好久，京东商城购买。...,2023-08-04 12:34:17,y***6,201,5,1,0,【驱蚊】便携喷雾 冰莲香型*1瓶,NaN
3,一直用的这个牌子，很好用，很喜欢，价格也实惠，味道很清香，也不是那么浓，夏天蚊子太多，必须要...,2023-07-19 05:12:38,独***9,201,5,0,0,【驱蚊】便携喷雾 清新花香*3瓶,NaN
4,六神驱蚊花露水真的太神奇了。看了很多驱蚊花露水，六神的特别好用。喷上花露水之后，蚊子躲得远远...,2023-07-28 11:08:17,蓝***咩,201,5,0,0,【驱蚊】便携喷雾 清新花香*3瓶,NaN
5,六神！又是一个神奇而久远的品牌！绝对是国产驱蚊产品的老字号了！这次买了这个花露水只要看到他有...,2023-07-29 23:21:22,醉***何,201,5,0,0,【驱蚊】便携喷雾 冰莲香型*1瓶,NaN
6,昨天中午下单，今天上午就收到了。京东上购物就是这么快和神奇。六神花露水气味很好，和描述的一样...,2023-07-30 10:50:45,o***k,201,5,0,0,【驱蚊】便携喷雾 冰莲香型*1瓶,NaN
7,看买了，这次买了好多哟，一大包的东西了，然后这个嗯花露水，六神花露水，经典的还有这个驱蚊花露...,2023-08-03 16:19:04,妮***g,201,5,0,0,【驱蚊】便携喷雾 冰莲香型*1瓶,NaN
8,六神止痒花露水，止痒效果挺快速的，一直用这款六神止痒花露水，夏天到了，蚊子咬了疙瘩，确实很难...,2023-07-19 14:22:09,彬***b,201,5,0,0,【快速止痒】便携喷雾 草本*1瓶,NaN
9,外观:外观看起来很高大上，质感很好，手感也很好\n质地:跟以前一样的，用起来很方便\n气味:...,2023-07-11 11:23:21,东***墨,201,5,0,0,【爆凉】便携喷雾 冰爽提神*1瓶,NaN


In [25]:
pd_reader.tail(8)    # 查看后几行信息，默认为5

,content,creationTime,nickname,plusAvailable,score,usefulVoteCount,replyCount,productColor,productSize
92,夏天必备，在外面执勤的兄弟伙些说夏天蚊虫太多了，需要花露水，我的第一印象就想到了六神花露水，...,2023-07-28 13:06:27,5***9,201,5,0,0,【止痒】便携喷雾 花漾清新*1瓶,NaN
93,已经第二次购买了！非常不错的强劲凉爽！我很奇怪的发现，我们这里很多大型的超市六神花露水品种也...,2023-07-15 01:39:45,8***汪,201,5,0,0,【爆凉】便携喷雾 冰爽提神*1瓶,NaN
94,一次性买了很多瓶，用于发放给员工夏季防暑降温除蚊的，性价比较高，比超市便宜，物流还超快，次日...,2023-07-26 11:36:40,星***了,0,5,0,0,【驱蚊】便携喷雾 清新花香*3瓶,NaN
95,这种驱蚊花露水，这几年一直在使用，驱蚊效果很好，喷上后，蚊子?只会在身边飞，却不敢落在身上。...,2023-07-09 14:14:09,s***m,201,5,0,0,【驱蚊】便携喷雾 清新花香*1瓶,NaN
96,老牌子国货了，从小用到大，品质一直未变，还是原来的配方，驱蚊效果杠杠的好，出门旅游避免的蚊虫...,2023-07-29 09:56:41,大***2,201,5,0,0,【驱蚊】便携喷雾 冰莲香型*3瓶,NaN
97,物流速度很快，价格也很实惠。喷后很凉爽很舒服，止痒效果也很不错，驱蚊也还行，闻起来也很香包装...,2023-07-09 22:33:43,A***神,201,5,0,0,【止痒】便携喷雾 花漾清新*1瓶,NaN
98,快递收到了，快递员很辛苦的，送货上门，谢谢了！这个驱蚊花露水，可以防蚊子咬着，咬着了也可以喷...,2023-07-24 17:43:25,g***8,201,5,0,0,【驱蚊】便携喷雾 清新花香*1瓶,NaN
99,六神花露水，使用若干年了，信赖的品牌。家里蚊子多，使用此款驱蚊花露水特别有效，喷雾型的节约而...,2023-07-08 10:43:22,湘***哥,201,5,0,0,【驱蚊】便携喷雾 清新花香*1瓶,NaN


<center><big><b>END</b></big></center>